In [25]:
# https://kanoki.org/2019/02/26/compare-two-excel-files-for-difference-using-python/

# Stock trade verification code

import numpy as np
import pandas as pd
import datetime

In [26]:
# 현재 날짜 가져오기
current_date = datetime.datetime.now()

# yyyymmdd 형식으로 변환
# td = current_date.strftime("%Y%m%d")
# td_tr1 = current_date.strftime("%m월").lstrip("0")
# td_tr2 = current_date.strftime("%d일").lstrip("0")
# td_tr = td_tr1 + td_tr2

td = "20230607"
td_tr = "6월7일"

print("Today : " + td)


Today : 20230607


In [27]:
oms_df = pd.read_excel("Z:/02.펀드/019. 일간매매내역/" + td + "_oms.xls", )
trader_df = pd.read_excel("Z:/02.펀드/019. 일간매매내역/" + td_tr + " 거래.xlsx", )

In [28]:
# Short sell, Buy cover, Buy, Sell 매매구분을 위한 열 병합 및 값 변환
oms_df1 = oms_df[["펀드", "종목명", "매매유형", "매매구분", "체결수량", "체결단가", "체결금액"]]

oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)

oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])

oms_df1 = oms_df1.drop(oms_df1.columns[[2]], axis=1)

# oms_df1 = oms_df1.replace(1, np.nan)
oms_df1 = oms_df1.dropna()

col_to_move = oms_df1.pop('매매구분')

oms_df1.insert(2, '매매구분', col_to_move)

C:\Users\jtlee\AppData\Local\Temp\ipykernel_26320\1968330389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)
C:\Users\jtlee\AppData\Local\Temp\ipykernel_26320\1968330389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])


#둘 다 키값을 종목명(Key1) and 체결수량(Key2)으로 잡고 정렬한 다음 데이터프레임의 밸류들이 일치하는지 확인

In [29]:
oms_df1

,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,공모주1호,기아,Buy cover,458,85365,39097200
1,공모주1호,대주전자재료,Short sell,100,97569,9756900
2,공모주1호,KODEX코스닥150선물인버스,Buy,12700,3943,50072370
3,공모주1호,제이오,Buy,300,32699,9809650
4,코스닥벤처,기아,Buy cover,456,85598,39032800
5,코스닥벤처,SFA반도체,Short sell,2000,5689,11377690
6,코스닥벤처,하나마이크론,Short sell,600,17066,10239840
7,코스닥벤처,제넥신,Short sell,586,12290,7201980
8,코스닥벤처,KODEX코스닥150선물인버스,Buy,12700,3943,50078640
9,코스닥벤처,자람테크놀로지,Sell,775,37255,28872550


In [30]:
oms_df2 = oms_df1.set_index(['펀드', '종목명', '매매구분'])

oms_df3 = oms_df2.sort_index(axis=0)

oms_df3 = oms_df3[(oms_df3 != 0).all(axis=1)]

oms_df4 = oms_df3.groupby(['펀드', '종목명', '매매구분']).agg({'체결수량': 'sum', '체결단가': 'mean', '체결금액': 'sum'})

In [31]:
oms_df4

체결수량      체결단가      체결금액
펀드    종목명              매매구분                                 
공모주1호 KODEX코스닥150선물인버스 Buy         12700    3943.0  50072370
      기아               Buy cover     458   85365.0  39097200
      대주전자재료           Short sell    100   97569.0   9756900
      제이오              Buy           300   32699.0   9809650
멀티전략  POSCO홀딩스         Buy cover      30  389000.0  11670000
      SFA반도체           Short sell   2000    5692.0  11383250
      기아               Buy cover     440   85402.0  37577000
      레이크머티리얼즈         Short sell    700   15530.0  10871070
      원익IPS            Short sell    800   31759.0  25407150
코스닥벤처 KODEX코스닥150선물인버스 Buy         12700    3943.0  50078640
      SFA반도체           Short sell   2000    5689.0  11377690
      기아               Buy cover     456   85598.0  39032800
      유안타제10호스팩        Sell          581    2164.0   1257065
      자람테크놀로지          Sell          775   37255.0  28872550
      제넥신              Short sell    586   12290.0   7201980
      하나마이크론           Short sell    600   17066.0  10239840

In [32]:
trader_df1 = trader_df[["펀드", "종목명", "매매구분", "체결수량", "체결단가", "체결금액"]]

In [33]:
trader_df1 = trader_df1.dropna()

trader_df2 = trader_df1.set_index('펀드', '종목명', '매매구분')

trader_df3 = trader_df2.groupby(['펀드', '종목명', '매매구분']).agg({'체결수량': 'sum', '체결단가': 'mean', '체결금액': 'sum'})

C:\Users\jtlee\AppData\Local\Temp\ipykernel_26320\2518177081.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  trader_df2 = trader_df1.set_index('펀드', '종목명', '매매구분')


In [34]:
trader_df3 = trader_df3.sort_index(axis=0)

In [35]:
trader_df3 = trader_df3.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})
oms_df4 = oms_df4.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})

In [36]:
trader_df3['체결금액'] = trader_df3['체결금액'].abs()

trader_df3 = trader_df3.reset_index()
oms_df4 = oms_df4.reset_index()

try:
    trader_df3 = trader_df3[trader_df3['종목명'] != 'KOSPI200F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != 'KOSDAQ150F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스피200F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스닥150F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스피F200']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스닥F150']
except:
    pass

In [37]:
# trader_df3.compare(oms_df3)

In [38]:
trader_df3 = trader_df3.reset_index()
oms_df4 = oms_df4.reset_index()

In [39]:
trader_df3

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주1호,KODEX코스닥150선물인버스,Buy,12700,3943,50076100
1,1,공모주1호,기아,Buy cover,458,85365,39097170
2,2,공모주1호,대주전자재료,Short sell,100,97569,9756900
3,3,공모주1호,제이오,Buy,300,32699,9809700
4,4,멀티전략,POSCO홀딩스,Buy cover,30,389000,11670000
5,5,멀티전략,기아,Buy cover,440,85402,37576880
6,6,멀티전략,레이크머티리얼즈,Short sell,700,15530,10871000
7,7,멀티전략,원익IPS,Short sell,800,31759,25407200
8,8,코스닥벤처,KODEX코스닥150선물인버스,Buy,12700,3943,50076100
9,9,코스닥벤처,기아,Buy cover,456,85598,39032688


In [40]:
oms_df4

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주1호,KODEX코스닥150선물인버스,Buy,12700,3943,50072370
1,1,공모주1호,기아,Buy cover,458,85365,39097200
2,2,공모주1호,대주전자재료,Short sell,100,97569,9756900
3,3,공모주1호,제이오,Buy,300,32699,9809650
4,4,멀티전략,POSCO홀딩스,Buy cover,30,389000,11670000
5,5,멀티전략,SFA반도체,Short sell,2000,5692,11383250
6,6,멀티전략,기아,Buy cover,440,85402,37577000
7,7,멀티전략,레이크머티리얼즈,Short sell,700,15530,10871070
8,8,멀티전략,원익IPS,Short sell,800,31759,25407150
9,9,코스닥벤처,KODEX코스닥150선물인버스,Buy,12700,3943,50078640


In [41]:
result_df = oms_df4

In [42]:
result_df[['종목명']] = oms_df4[['종목명']]
result_df[['매매구분']] = oms_df4[['매매구분']]
result_df[['체결수량']] = oms_df4[['체결수량']] - trader_df3[['체결수량']]
result_df[['체결단가']] = oms_df4[['체결단가']] - trader_df3[['체결단가']]
result_df[['체결금액']] = oms_df4[['체결금액']] - trader_df3[['체결금액']]

In [43]:
result_df

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주1호,KODEX코스닥150선물인버스,Buy,0.0,0.0,-3730.0
1,1,공모주1호,기아,Buy cover,0.0,0.0,30.0
2,2,공모주1호,대주전자재료,Short sell,0.0,0.0,0.0
3,3,공모주1호,제이오,Buy,0.0,0.0,-50.0
4,4,멀티전략,POSCO홀딩스,Buy cover,0.0,0.0,0.0
5,5,멀티전략,SFA반도체,Short sell,1560.0,-79710.0,-26193630.0
6,6,멀티전략,기아,Buy cover,-260.0,69872.0,26706000.0
7,7,멀티전략,레이크머티리얼즈,Short sell,-100.0,-16229.0,-14536130.0
8,8,멀티전략,원익IPS,Short sell,-11900.0,27816.0,-24668950.0
9,9,코스닥벤처,KODEX코스닥150선물인버스,Buy,12244.0,-81655.0,11045952.0


In [44]:
# result_df.to_excel('Z:/02.펀드/019. 일간매매내역/recon_result/' + td + "_recon_result.xlsx", index=True)